# Transfer Learning 활용하기

## 라이브러리 가져오기


In [1]:
import tensorflow as tf
import matplotlib.pylab as plt

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np

## ImageNet에 대해 잘 훈련된 Feature Extractor


### VGG16의 Classifier 형태 확인

본 실습에서는 tf.keras.applications 에 제공되는 모델 중 2일차에 직접 작성도 하였던 VGG16을 우선 가져와 summary를 찍어보겠습니다.


In [3]:
model_vgg = tf.keras.applications.VGG16(weights=None)
model_vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

마지막 Classifier 부분의 형태를 확인할 수 있는데,

> Flatten()  
> Dense(4096)  
> Dense(4096)  
> Dense(1000)  

으로 작성되어 있는 것을 알 수 있습니다.

### include_top 옵션
tf.keras.applications에는 "include_top"이라는 옵션이 있으며, 이 옵션을 False로 두면 Classifier 부분을 날려주는 것을 확인할 수 있습니다. 아래 셀을 실행시켜 summary()를 확인해주세요.

In [4]:
model_vgg_notop = tf.keras.applications.VGG16(include_top=False, weights=None)
model_vgg_notop.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

확인해보시면, 정확하게 Flatten()부터 제거되어 있는 것을 확인하실 수 있습니다.

하지만 위에서 가져온 모델은 학습이 진행되지 않은 randomly initialized model입니다.

### Feature Extractor (Topless Model) 가져오기
이제 Transfer Learning 본연의 목적에 맞게 잘 학습된 모델을 활용해보기 위해 ImageNet에 대해 학습된 모델을 가져와보겠습니다.  
이 때 들어갈 입력도 VGG16이 훈련된 224, 224에 맞춰주도록 하겠습니다.

In [5]:
feature_extractor = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
feature_extractor.summary()

58892288/58889256 [==============================] - 10s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_____________________________________________________________

## Dataset 준비 - Food-5K Dataset

실습을 위해 Dataset을 준비해보겠습니다.

우선 Food-5k 데이터셋을 다운로드 받습니다.  
Food-5k는 5000장으로 이루어진 Food or None Food의 라벨을 가진 데이터셋입니다.

- **이미지의 이름 형식은 다음과 같으며**
> {ClassID}_{ImageID}.jpg
  
  
- **ClassID와 ImageID는 아래와 같습니다.**
> ClassID: 0 or 1; 0 means non-food and 1 means food.  
> ImageID: ID of the image within the class.

### Dataset 다운로드 및 디렉토리 구조 변경

In [ ]:
!wget https://s3-us-west-2.amazonaws.com/static.pyimagesearch.com/food-datasets/Food-5K.zip

flow_from_directory를 활용하기 위해 디렉토리 계층구조를 재정비 합니다.

In [ ]:
!mkdir dataset
!unzip -q Food-5K.zip -d dataset
!rm -r dataset/__MACOSX/

In [ ]:
!ls dataset/

In [ ]:
!mkdir -p dataset/training/nonfood
!mkdir -p dataset/training/food
!mkdir -p dataset/validation/nonfood
!mkdir -p dataset/validation/food
!mkdir -p dataset/evaluation/nonfood
!mkdir -p dataset/evaluation/food

In [ ]:
!mv dataset/training/0* dataset/training/nonfood/
!mv dataset/training/1* dataset/training/food/
!mv dataset/validation/0* dataset/validation/nonfood
!mv dataset/validation/1* dataset/validation/food
!mv dataset/evaluation/0* dataset/evaluation/nonfood
!mv dataset/evaluation/1* dataset/evaluation/food

결과적으로 데이터셋의 구조는 아래와 같아집니다.

<pre>
<b>dataset</b>
|__ <b>training</b>
    |______ <b>nonfood</b>: [0_0.jpg, 0_1.jpg, 0_2.jpg ....]
    |______ <b>food</b>: [1_0.jpg, 1_1.jpg, 1_2.jpg ....]
|__ <b>validation</b>
    |______ <b>nonfood</b>: [0_0.jpg, 0_1.jpg, 0_2.jpg ....]
    |______ <b>food</b>: [1_0.jpg, 1_1.jpg, 1_2.jpg ....]
|__ <b>evaluation</b>
    |______ <b>nonfood</b>: [0_0.jpg, 0_1.jpg, 0_2.jpg ....]
    |______ <b>food</b>: [1_0.jpg, 1_1.jpg, 1_2.jpg ....]
</pre>

또한, 이미지를 직접 확인해보면 다음과 같습니다.

In [ ]:
img = plt.imread('dataset/training/food/1_0.jpg')
plt.imshow(img)

In [ ]:
img = plt.imread('dataset/training/nonfood/0_0.jpg')
plt.imshow(img)

### Image Generator 및 Flow를 통한 데이터 준비

In [ ]:
train_dir = 'dataset/training/'
validation_dir = 'dataset/validation/'

tr_food = 'dataset/training/food/'
tr_nfood = 'dataset/training/nonfood/'
va_food = 'dataset/validation/food/'
va_nfood = 'dataset/validation/nonfood/'

print(len(os.listdir(tr_food)))
print(len(os.listdir(tr_nfood)))
print(len(os.listdir(va_food)))
print(len(os.listdir(va_nfood)))

Training Data 3000장 중 각 클래스 별 1500장,  
Validation Data 1000장 중 각 클래스 별 500장  
으로 구성되어 있는 것을 알 수 있습니다.

In [ ]:
batch_size = 128
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

**결과적으로 food는 0으로, nonfood는 1로 지정된 것을 확인할 수 있습니다.**

In [ ]:
train_data_gen.class_indices

## Transfer Learning

### Image Batch에 대한 Classifier 실행

In [ ]:
sample_training_images, sample_training_labels = next(train_data_gen)

In [ ]:
result_batch = feature_extractor.predict(sample_training_images)
result_batch.shape

각각의 이미지마다 (7, 7, 512)인 벡터가 반환되는 것을 확인할 수 있습니다.

Feature extractor layer에 있는 변수들을 학습 불가능하도록 만들면, 학습은 오직 새로운 classifier layer에만 가능하게 됩니다.

In [ ]:
feature_extractor.trainable = False

### Classifier를 붙이기

이제 `tf.keras.Sequential` 모델에 Flatten을 적용하고, 새로운 classifier layer를 추가합니다.

In [6]:
model = tf.keras.Sequential([
  feature_extractor,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               12845568  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 27,560,769
Trainable params: 27,560,769
Non-trainable params: 0
_________________________________________________________________


이제 완성된 모델에 샘플 이미지 Batch를 넣어보면,

In [ ]:
predictions = model(sample_training_images)

결과적으로 1의 출력이 나온다는 것을 알 수 있습니다.

In [ ]:
predictions.shape

### 모델 학습하기

학습 과정을 만들기 위해 모델을 컴파일 합니다.

In [ ]:
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['acc'])

5 Epoch 정도 학습을 진행시켜가며 경과를 지켜보도록 하겠습니다.

In [ ]:
epochs = 5

In [ ]:
history = model.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen,
)

첫 번째 Epoch부터 90%를 상회하는 Validation Accuracy를 확인하실 수 있으며,  
5 Epoch 정도 수행되었을 때 약 96% 정도로 수렴합니다.

### 학습 결과 시각화

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 학습된 모델로 Inference 해보기

음식과 음식이 아닌 사물에 대한 이미지를 적당히 다운받습니다.

In [ ]:
!wget https://media-cdn.tripadvisor.com/media/photo-s/16/5c/a9/7d/lahore-food.jpg -O food.jpg
!wget https://www.oxfordsaudia.com/wp-content/uploads/2018/07/banner-airplane-628x439.jpg -O nonfood.jpg

이미지를 학습시킨 모델에 맞추어 rescale 후 (1, 224, 224, 3)로 변환시킨다.

In [ ]:
# keras 라이브러리를 이용한 방법
def img2input_keras(path, target_size):
  img_show = plt.imread(path)
  plt.imshow(img_show)
  tmp = tf.keras.preprocessing.image.load_img(path, target_size=target_size)
  tmp = tf.keras.preprocessing.image.img_to_array(tmp)
  tmp = tmp/255.
  tmp = np.expand_dims(tmp, axis=0)
  print(tmp.shape)
  return tmp

In [ ]:
food_input = img2input_keras('food.jpg', (224, 224))

In [ ]:
print('food image -> ', model.predict(food_input))
print(model.predict(food_input), 'is almost', round(model.predict(food_input)[0][0]))
print(train_data_gen.class_indices)

In [ ]:
nfood_input = img2input_keras('nonfood.jpg', (224, 224))

In [ ]:
print('nonfood image -> ', model.predict(nfood_input))
print(model.predict(nfood_input), 'is almost', round(model.predict(nfood_input)[0][0]))
print(train_data_gen.class_indices)

## Model Export 및 Load하기

학습시킨 모델을 내보내었다가 다시 불러들여 그 값을 비교해보도록 하겠습니다.

In [ ]:
model_name = 'mymodel'

export_path = "/tmp/saved_models/"+model_name
model.save(export_path, save_format='tf')

export_path

Export된 모델을 다시 로딩할 수 있고, 이는 동일한 결과를 보여줍니다.

In [ ]:
reloaded = tf.keras.models.load_model(export_path)

In [ ]:
result_batch = model.predict(sample_training_images)
reloaded_result_batch = reloaded.predict(sample_training_images)

In [ ]:
abs(reloaded_result_batch - result_batch).max()